In [148]:
import pandas as pd
import numpy as np
import time
import re
import copy
import random
import os
from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


In [326]:
# 프록시 주소를 중간중간 변경하여 로봇에 탐지되는 것을 회피하기 위한 코드
# 시스템 프록시 설정 읽기 (예: Windows 환경 변수 사용)

proxy = os.environ.get('http_proxy') or os.environ.get('https_proxy')

options = webdriver.ChromeOptions()
options.add_argument(f'--proxy-server={proxy}')

driver = webdriver.Chrome(options=options)

In [286]:
df = pd.read_excel('data/justwatch_first.xlsx', index_col=0)
df.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021,https://www.imdb.com/title/tt10919420/
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Mini Series 2022,https://www.imdb.com/title/tt13589004/
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020,https://www.imdb.com/title/tt12079212/
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021,https://www.imdb.com/title/tt12809988/
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020,https://www.imdb.com/title/tt11612120/


In [3]:
df['Production country'].unique()

array(['대한민국', '벨기에', '대만', '미국', '덴마크', '독일', '일본', '태국', '캐나다', '그리스',
       '영국', 'China', '스페인', '미국, 영국',
       '아이슬란드, 덴마크, 핀란드, 스웨덴, 노르웨이, 독일, 프랑스, 영국', '미국, 프랑스', '호주', '스웨덴',
       '이탈리아', '케냐', 'WEBAPP_COUNTRY_VN', '인도', '호주, 미국', '스위스', '필리핀 제도',
       '영국, 미국', '남아프리카공화국', '폴란드', '브라질', '미국, 캐나다', '콜롬비아', '터키',
       '룩셈부르크', '핀란드', '이스라엘', '프랑스', '멕시코', '벨기에, 네덜란드', '덴마크, 노르웨이',
       '노르웨이', '러시아', '미국, 일본', '영국, 프랑스', '요르단', '프랑스, 미국', '캐나다, 영국',
       '태국, 미국', '인도네시아', '사우디아라비아', '영국, 인도', 'Arjantin', '아이슬란드',
       '오스트리아', '포르투갈', '프랑스, 독일, 스페인, 영국', '아일랜드, 영국', '캐나다, 프랑스, 독일',
       '이스라엘, 노르웨이', '대한민국, 미국', '미국, 대한민국', '네덜란드', '멕시코, 스페인', '홍콩',
       '칠레', '뉴질랜드, 호주', '나이지리아', '이집트', '일본, 미국', '미국, 멕시코', '쿠웨이트',
       '나이지리아, 미국', '말레이시아, 대만', '사우디아라비아, 인도', '벨기에, 룩셈부르크',
       '사우디아라비아, 튀니지, 이집트, 모로코, 레바논', '독일, 오스트리아', '체코 공화국, 독일',
       '영국, 스페인', nan, '스페인, 브라질', '레바논'], dtype=object)

In [4]:
countries_dict = {
    '대한민국': 'Korea',
    '벨기에': 'Belgium',
    '대만': 'Taiwan',
    '미국': 'US',
    '덴마크': 'Denmark',
    '독일': 'Germany',
    '일본': 'Japan',
    '태국': 'Thailand',
    '캐나다': 'Canada',
    '그리스': 'Greece',
    '영국': 'United Kingdom',
    'China': 'China',
    '스페인': 'Spain',
    '미국, 영국': 'US, United Kingdom',
    '아이슬란드, 덴마크, 핀란드, 스웨덴, 노르웨이, 독일, 프랑스, 영국': 'Iceland, Denmark, Finland, Sweden, Norway, Germany, France, United Kingdom',
    '미국, 프랑스': 'US, France',
    '호주': 'Australia',
    '스웨덴': 'Sweden',
    '이탈리아': 'Italy',
    '케냐': 'Kenya',
    'WEBAPP_COUNTRY_VN': 'Vietnam',
    '인도': 'India',
    '호주, 미국': 'Australia, US',
    '스위스': 'Switzerland',
    '필리핀 제도': 'Philippines',
    '영국, 미국': 'United Kingdom, US',
    '남아프리카공화국': 'South Africa',
    '폴란드': 'Poland',
    '브라질': 'Brazil',
    '미국, 캐나다': 'US, Canada',
    '콜롬비아': 'Colombia',
    '터키': 'Turkey',
    '룩셈부르크': 'Luxembourg',
    '핀란드': 'Finland',
    '이스라엘': 'Israel',
    '프랑스': 'France',
    '멕시코': 'Mexico',
    '벨기에, 네덜란드': 'Belgium, Netherlands',
    '덴마크, 노르웨이': 'Denmark, Norway',
    '노르웨이': 'Norway',
    '러시아': 'Russia',
    '미국, 일본': 'US, Japan',
    '영국, 프랑스': 'United Kingdom, France',
    '요르단': 'Jordan',
    '프랑스, 미국': 'France, US',
    '캐나다, 영국': 'Canada, United Kingdom',
    '태국, 미국': 'Thailand, US',
    '인도네시아': 'Indonesia',
    '사우디아라비아': 'Saudi Arabia',
    '영국, 인도': 'United Kingdom, India',
    'Arjantin': 'Argentina',
    '아이슬란드': 'Iceland',
    '오스트리아': 'Austria',
    '포르투갈': 'Portugal',
    '프랑스, 독일, 스페인, 영국': 'France, Germany, Spain, United Kingdom',
    '아일랜드, 영국': 'Ireland, United Kingdom',
    '캐나다, 프랑스, 독일': 'Canada, France, Germany',
    '이스라엘, 노르웨이': 'Israel, Norway',
    '대한민국, 미국': 'South Korea, US',
    '미국, 대한민국': 'US, South Korea',
    '네덜란드': 'Netherlands',
    '멕시코, 스페인': 'Mexico, Spain',
    '홍콩': 'Hong Kong',
    '칠레': 'Chile',
    '뉴질랜드, 호주': 'New Zealand, Australia',
    '나이지리아': 'Nigeria',
    '이집트': 'Egypt',
    '일본, 미국': 'Japan, US',
    '미국, 멕시코': 'US, Mexico',
    '쿠웨이트': 'Kuwait',
    '나이지리아, 미국': 'Nigeria, US',
    '말레이시아, 대만': 'Malaysia, Taiwan',
    '사우디아라비아, 인도': 'Saudi Arabia, India',
    '벨기에, 룩셈부르크': 'Belgium, Luxembourg',
    '사우디아라비아, 튀니지, 이집트, 모로코, 레바논': 'Saudi Arabia, Tunisia, Egypt, Morocco, Lebanon',
    '독일, 오스트리아': 'Germany, Austria',
    '체코 공화국, 독일': 'Czech Republic, Germany',
    '영국, 스페인': 'United Kingdom, Spain',
    '스페인, 브라질': 'Spain, Brazil',
    '레바논': 'Lebanon'
}

In [327]:
driver = webdriver.Chrome()
driver.get('https://www.google.com/')

In [ ]:
df.loc[0]['title'], df.loc[0]['year'], df.loc[0]['Production country']

('Ojingeo Geim', 2021, '대한민국')

In [ ]:
title = df.loc[0]['title']
year = df.loc[0]['year']
country = countries_dict[df.loc[0]['Production country']]

In [ ]:
driver.find_element(By.CSS_SELECTOR, '#APjFqb').send_keys(f'{title}({year}) {country} TV show IMDb')

In [ ]:
driver.find_element(By.CSS_SELECTOR, '#APjFqb').send_keys(Keys.ENTER)

In [ ]:
driver.find_element(By.CSS_SELECTOR, '#rso > div:nth-child(1) > div > div > div > div.kb0PBd.cvP2Ce.A9Y9g.jGGQ5e > div > div > span > a').get_attribute('href')

'https://www.imdb.com/title/tt10919420/'

In [ ]:
driver.find_element(By.CSS_SELECTOR, 'a[jsname=UWckNb]').get_attribute('href')


'https://www.imdb.com/title/tt23581746/'

In [ ]:
driver.find_element(By.CSS_SELECTOR, 'span > a > h3').text
#rso > div:nth-child(1) > div > div > div > div > div > div > div > div.yuRUbf > div > span > a > h3

'The Flash (TV Series 2014–2023) - IMDb'

In [46]:
english_title = []
original_title = []

In [46]:
null_index = df[df['original_title'].isna()].index

In [ ]:
# 구글에서 justwatch 페이지 가져오기

for idx, row in tqdm_notebook(df.iterrows()):
    if idx not in null_index:
        title = row['original_title']
    else:
        title = row['title']
    year = row['year']
    country = countries_dict[row['Production country']]

    driver.find_element(By.CSS_SELECTOR, '#APjFqb').send_keys(f'{title}({year}) {country} TV show justwatch')
    driver.find_element(By.CSS_SELECTOR, '#APjFqb').send_keys(Keys.ENTER)

    time.sleep(1)

    if 'justwatch' in driver.find_element(By.CSS_SELECTOR, 'a[jsname=UWckNb]').get_attribute('href'):
        driver.find_element(By.CSS_SELECTOR, 'div > div > span > a > h3').click()
    else:
        original_title.append('검색 결과 부정확')
        english_title.append('검색 결과 부정확')
        driver.find_element(By.CSS_SELECTOR, '#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef > div > span > svg > path').click()
        continue
    
    time.sleep(1)

    # justwatch에서 'too many request' 오류가 날 경우를 대비해 except문 추가
    try:
        if year == int(driver.find_element(By.CSS_SELECTOR, 'div.title-block > div > span').text[1:5]):
            english_title.append(driver.find_element(By.CSS_SELECTOR, 'div.title-block > div > h1').text)
            try:
                original_title.append(driver.find_element(By.CSS_SELECTOR, 'div.title-block > h3').text.split(': ', 1)[1])
            except:
                original_title.append('오리지날 타이틀 없음')
        else:
            original_title.append('검색 결과 부정확')
            english_title.append('검색 결과 부정확')
    except:
        original_title.append('검색 결과 부정확')
        english_title.append('검색 결과 부정확')

    driver.back()
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, '#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef > div > span > svg > path').click()

In [287]:
df['add_original_title'] = original_title
df['add_english_title'] = english_title

In [ ]:
# 'too many request'로 거부되어 누락된 정보 재입력

for idx, row in tqdm_notebook(df[df['add_english_title'] == '검색 결과 부정확'].iterrows()):
    if idx not in null_index:
        title = row['original_title']
    else:
        title = row['title']
    year = row['year']
    country = countries_dict[row['Production country']]

    driver.find_element(By.CSS_SELECTOR, '#APjFqb').send_keys(f'{title}({year}) {country} TV show justwatch')
    driver.find_element(By.CSS_SELECTOR, '#APjFqb').send_keys(Keys.ENTER)

    time.sleep(1)

    if 'justwatch' in driver.find_element(By.CSS_SELECTOR, 'a[jsname=UWckNb]').get_attribute('href'):
        driver.find_element(By.CSS_SELECTOR, 'div > div > span > a > h3').click()
    else:
        df.loc[idx,'add_original_title'] = '검색 결과 부정확'
        df.loc[idx,'add_english_title'] = '검색 결과 부정확'
        driver.find_element(By.CSS_SELECTOR, '#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef > div > span > svg > path').click()
        continue
    
    time.sleep(1)

    # justwatch에서 'too many request' 오류가 날 경우를 대비해 except문 추가
    try:
        if year == int(driver.find_element(By.CSS_SELECTOR, 'div.title-block > div > span').text[1:5]):
            df.loc[idx,'add_english_title'] = driver.find_element(By.CSS_SELECTOR, 'div.title-block > div > h1').text
            try:
                df.loc[idx,'add_original_title'] = driver.find_element(By.CSS_SELECTOR, 'div.title-block > h3').text.split(': ', 1)[1]
            except:
                df.loc[idx,'add_original_title'] = '오리지날 타이틀 없음'
        else:
            df.loc[idx,'add_original_title'] = '검색 결과 부정확'
            df.loc[idx,'add_english_title'] = '검색 결과 부정확'
    except:
        df.loc[idx,'add_original_title'] = '검색 결과 부정확'
        df.loc[idx,'add_english_title'] = '검색 결과 부정확'

    driver.back()
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, '#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef > div > span > svg > path').click()

In [289]:
df.to_excel('add_eng_title.xlsx')